# Micro-Climate Building Context Visualization 

# Dependencies

In [ ]:
!pip install geopandas

In [ ]:
!pip install rtree

In [3]:
import geopandas as gpd
import pandas as pd
import rtree
from shapely.geometry import Polygon, MultiPolygon, Point
import shapely.wkt as wkt
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import csv
import os

# Swiss Dwelling Data Preparation

In [4]:
#mount google drive
drive.mount ('/content/gdrive', force_remount=True)
folder = "/content/gdrive/My Drive/Archilyse/context/v2.2.1"
os.chdir(folder)

Mounted at /content/gdrive


In [5]:
df_simulation = pd.read_csv("simulations.csv")

In [6]:
df_geometry = pd.read_csv("geometries.csv")

In [ ]:
list(df_simulation.columns)

keeping the needed columns from both files from the dataset 

In [7]:
df_simulation = df_simulation[['site_id','building_id','floor_id','unit_id','area_id','apartment_id','sun_201803210800_max','sun_201803211600_max', 'view_sky_max','view_greenery_max', 'window_noise_traffic_day_max']]

In [ ]:
df_simulation.info()

In [9]:
df_geometry_clean = df_geometry[['area_id','entity_type','entity_subtype','geometry']]

In [ ]:
df_geometry_clean.head()

In [ ]:
df_geometry_clean.info()

merging the geometry and simulation files based on the shared column (area_id)

In [12]:
df = pd.merge(df_simulation, df_geometry_clean, on ='area_id')

In [ ]:
df.info()

In [ ]:
#cleaning the merged dataframe by dropping dublucate or incomplete rows
df_clean = df.drop_duplicates()
df_clean.info()

In [15]:
df = df_clean.dropna()

In [16]:
#only keeping the area type in the merged dataset
df = df[df['entity_type'] == "area"]

from now on, df is the clean and merged dataframe

In [ ]:
df.info()

In [ ]:
#number of unique ids
print(df.nunique())

In [19]:
#Constructing GeoDataFrame from a pandas DataFrame with a column of WKT geometries:
gs = gpd.GeoSeries.from_wkt(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry=gs, crs=None)

In [ ]:
#plotting the whole dataframe
fig = plt.figure(figsize=(30, 18))
ax2 = gdf.plot('entity_subtype', cmap = "plasma", figsize=(30,30), legend=True)
ax2.figure.savefig("entity_subtype")

# Specific id 

In [ ]:
#finding data about specific ids
test_floor = df.loc[df['floor_id'] == 987]

# Normalize using Min/Max Normalization.
test_floor['sun_201803210800_max']=(test_floor['sun_201803210800_max']-test_floor['sun_201803210800_max'].min())/(test_floor['sun_201803210800_max'].max()-test_floor['sun_201803210800_max'].min())
test_floor['sun_201803211600_max']=(test_floor['sun_201803211600_max']-test_floor['sun_201803211600_max'].min())/(test_floor['sun_201803211600_max'].max()-test_floor['sun_201803211600_max'].min())
test_floor['view_sky_max']=(test_floor['view_sky_max']-test_floor['view_sky_max'].min())/(test_floor['view_sky_max'].max()-test_floor['view_sky_max'].min())
test_floor['view_greenery_max']=(test_floor['view_greenery_max']-test_floor['view_greenery_max'].min())/(test_floor['view_greenery_max'].max()-test_floor['view_greenery_max'].min())
test_floor['window_noise_traffic_day_max']=(test_floor['window_noise_traffic_day_max']-test_floor['window_noise_traffic_day_max'].min())/(test_floor['window_noise_traffic_day_max'].max()-test_floor['window_noise_traffic_day_max'].min())

test_floor.head()

In [ ]:
#first change dataframe to geopandas dataframe
gdf_test_floor = gpd.GeoDataFrame(test_floor, geometry=gs, crs=None)
gdf_test_floor.plot(column='entity_subtype', cmap="tab20b", legend=True, figsize=(10,10))

# Context Visualization

this first example is a sample for visualizing the indicator circles for the sky view factor 
the areas with high value of sky view are found and the corresponding indicators are plotted on top of them based on the base radius 

In [ ]:
floor = gpd.GeoDataFrame(test_floor, geometry=gs, crs=None)

#finding areas with highest sky view factor, in this example, the first four 
max_ids_sky = test_floor.iloc[test_floor['view_sky_max'].argsort()[-4:]]
max_ids_sky = max_ids_sky.sort_values(by= 'view_sky_max', ascending=False)
gdf_sky_area = gpd.GeoDataFrame(max_ids_sky, geometry=gs, crs=None)

#indicator circle placement
gdf_sky_area['geometry'].centroid
bounds = floor.total_bounds
x_distance = bounds[2] - bounds[0]
y_distance = bounds[3] - bounds[1]
radius_base = 0.15*max(x_distance, y_distance)
radius = radius_base*(test_floor['view_sky_max'].nlargest(4))
circle = gdf_sky_area.centroid.buffer(radius, resolution=16)

gc = gpd.GeoSeries(circle)
gdf_circles = gpd.GeoDataFrame(circle, geometry=gc, crs=None)

#overlaying the circle on top of the floor layout
res_symdiff = gdf_circles.overlay(floor, how='identity')
ax = res_symdiff.plot(alpha= 0.5, color = 'Grey',figsize=(10,10))
floor.plot(ax=ax, facecolor='none');

here we have the same examle, but with all the seleceted environmental factors

In [ ]:
# Normalize using Min/Max Normalization.
test_floor['sun_201803210800_max']=(test_floor['sun_201803210800_max']-test_floor['sun_201803210800_max'].min())/(test_floor['sun_201803210800_max'].max()-test_floor['sun_201803210800_max'].min())
test_floor['sun_201803211600_max']=(test_floor['sun_201803211600_max']-test_floor['sun_201803211600_max'].min())/(test_floor['sun_201803211600_max'].max()-test_floor['sun_201803211600_max'].min())
test_floor['view_sky_max']=(test_floor['view_sky_max']-test_floor['view_sky_max'].min())/(test_floor['view_sky_max'].max()-test_floor['view_sky_max'].min())
test_floor['view_greenery_max']=(test_floor['view_greenery_max']-test_floor['view_greenery_max'].min())/(test_floor['view_greenery_max'].max()-test_floor['view_greenery_max'].min())
test_floor['window_noise_traffic_day_max']=(test_floor['window_noise_traffic_day_max']-test_floor['window_noise_traffic_day_max'].min())/(test_floor['window_noise_traffic_day_max'].max()-test_floor['window_noise_traffic_day_max'].min())
floor = gpd.GeoDataFrame(test_floor, geometry=gs, crs=None)

bounds = floor.total_bounds
x_distance = bounds[2] - bounds[0]
y_distance = bounds[3] - bounds[1]
radius_base = 0.15*max(x_distance, y_distance)

#number of indicator circles
ncircles_sky = 4
ncircles_daylight = 2
ncircles_greenery = 2
ncircles_noise = 2

#View Sky
max_ids_sky = test_floor.iloc[test_floor['view_sky_max'].argsort()[-ncircles_sky:]]
max_ids_sky = max_ids_sky.sort_values(by= 'view_sky_max', ascending=False)
gdf_sky_area = gpd.GeoDataFrame(max_ids_sky, geometry=gs, crs=None)

radius = radius_base*(test_floor['view_sky_max'].nlargest(ncircles_sky))
circle = gdf_sky_area.centroid.buffer(radius, resolution=16)

gc = gpd.GeoSeries(circle)
gdf_circles_sky = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
identity_1 = gdf_circles_sky.overlay(floor, how='identity')
ax = identity_1.plot(alpha= 0.5, color = 'Grey',figsize=(10,10))

#Daylight at 8
max_ids_daylight_8 = test_floor.iloc[test_floor['sun_201803210800_max'].argsort()[-ncircles_daylight:]]
max_ids_daylight_8 = max_ids_daylight_8.sort_values(by= 'sun_201803210800_max', ascending=False)
gdf_daylight_8_area = gpd.GeoDataFrame(max_ids_daylight_8, geometry=gs, crs=None)

radius = radius_base*(test_floor['sun_201803210800_max'].nlargest(ncircles_daylight))
circle = gdf_daylight_8_area.centroid.buffer(radius, resolution=16)

gc = gpd.GeoSeries(circle)
gdf_circles_daylight_8 = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
identity_2 = gdf_circles_daylight_8.overlay(floor, how='identity')
ax = identity_2.plot(ax=ax, alpha= 0.5, color = 'Yellow',figsize=(10,10))

#Daylight at 16
max_ids_daylight_16 = test_floor.iloc[test_floor['sun_201803211600_max'].argsort()[-ncircles_daylight:]]
max_ids_daylight_16 = max_ids_daylight_16.sort_values(by= 'sun_201803211600_max', ascending=False)
gdf_daylight_16_area = gpd.GeoDataFrame(max_ids_daylight_16, geometry=gs, crs=None)

radius = radius_base*(test_floor['sun_201803211600_max'].nlargest(ncircles_daylight))
circle = gdf_daylight_16_area.centroid.buffer(radius, resolution=16)

gc = gpd.GeoSeries(circle)
gdf_circles_daylight_16 = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
identity_3 = gdf_circles_daylight_16.overlay(floor, how='identity')
ax = identity_3.plot(ax=ax, alpha= 0.5, color = 'Yellow',figsize=(10,10))

#Greenery
max_ids_view_greenery = test_floor.iloc[test_floor['view_greenery_max'].argsort()[-ncircles_greenery:]]
max_ids_view_greenery = max_ids_view_greenery.sort_values(by= 'view_greenery_max', ascending=False)
gdf_view_greenery_area = gpd.GeoDataFrame(max_ids_view_greenery, geometry=gs, crs=None)

radius = radius_base*(test_floor['view_greenery_max'].nlargest(ncircles_greenery))
circle = gdf_view_greenery_area.centroid.buffer(radius, resolution=16)

gc = gpd.GeoSeries(circle)
gdf_circles_greenery = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
identity_4 = gdf_circles_greenery.overlay(floor, how='identity')
ax = identity_4.plot(ax=ax, alpha= 0.5, color = 'Green',figsize=(10,10))


#Noise
max_ids_noise = test_floor.iloc[test_floor['window_noise_traffic_day_max'].argsort()[-ncircles_noise:]]
max_ids_noise = max_ids_noise.sort_values(by= 'window_noise_traffic_day_max', ascending=False)
gdf_noise_area = gpd.GeoDataFrame(max_ids_noise, geometry=gs, crs=None)

radius = radius_base*(test_floor['window_noise_traffic_day_max'].nlargest(ncircles_noise))
circle = gdf_noise_area.centroid.buffer(radius, resolution=16)

gc = gpd.GeoSeries(circle)
gdf_circles_noise = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
identity_5 = gdf_circles_noise.overlay(floor, how='identity')
ax = identity_5.plot(ax=ax, alpha= 0.5, color = 'Red',figsize=(10,10))

identity_6 = floor.overlay(floor, how='identity')
ax = identity_6.plot(ax=ax, alpha= 0.5, color = 'White',figsize=(10,10))
floor.plot(ax=ax, facecolor='none');

For the whole (or selected part of) dataset

In [ ]:
#for the dataset
floors_unique = df.floor_id.unique()
floors_unique_df = pd.DataFrame(floors_unique, columns = ['floor_id'])

#number of indicator circles for each environmental factor
ncircles_sky = 4
ncircles_daylight = 2
ncircles_greenery = 2
ncircles_noise = 2

#by defining the range of x, it is possible to plot the micro-climatic context visualizations for the desired numbber
x = range(20,60)
for i in x:
  test_floor = df.loc[df['floor_id'] == floors_unique[i]]
  # Normalize using Min/Max Normalization.
  test_floor['sun_201803210800_max']=(test_floor['sun_201803210800_max']-test_floor['sun_201803210800_max'].min())/(test_floor['sun_201803210800_max'].max()-test_floor['sun_201803210800_max'].min())
  test_floor['sun_201803211600_max']=(test_floor['sun_201803211600_max']-test_floor['sun_201803211600_max'].min())/(test_floor['sun_201803211600_max'].max()-test_floor['sun_201803211600_max'].min())
  test_floor['view_sky_max']=(test_floor['view_sky_max']-test_floor['view_sky_max'].min())/(test_floor['view_sky_max'].max()-test_floor['view_sky_max'].min())
  test_floor['view_greenery_max']=(test_floor['view_greenery_max']-test_floor['view_greenery_max'].min())/(test_floor['view_greenery_max'].max()-test_floor['view_greenery_max'].min())
  test_floor['window_noise_traffic_day_max']=(test_floor['window_noise_traffic_day_max']-test_floor['window_noise_traffic_day_max'].min())/(test_floor['window_noise_traffic_day_max'].max()-test_floor['window_noise_traffic_day_max'].min())
  floor = gpd.GeoDataFrame(test_floor, geometry=gs, crs=None)
  
  bounds = floor.total_bounds
  x_distance = bounds[2] - bounds[0]
  y_distance = bounds[3] - bounds[1]
  radius_base = 0.15*max(x_distance, y_distance)

  #View Sky
  max_ids_sky = test_floor.iloc[test_floor['view_sky_max'].argsort()[-ncircles_sky:]]
  max_ids_sky = max_ids_sky.sort_values(by= 'view_sky_max', ascending=False)
  gdf_sky_area = gpd.GeoDataFrame(max_ids_sky, geometry=gs, crs=None)

  radius = radius_base*(test_floor['view_sky_max'].nlargest(ncircles_sky))
  circle = gdf_sky_area.centroid.buffer(radius, resolution=16)

  gc = gpd.GeoSeries(circle)
  gdf_circles_sky = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
  identity_1 = gdf_circles_sky.overlay(floor, how='identity')
  ax = identity_1.plot(alpha= 0.5, color = 'Grey',figsize=(10,10))

  #Daylight at 8
  max_ids_daylight_8 = test_floor.iloc[test_floor['sun_201803210800_max'].argsort()[-ncircles_daylight:]]
  max_ids_daylight_8 = max_ids_daylight_8.sort_values(by= 'sun_201803210800_max', ascending=False)
  gdf_daylight_8_area = gpd.GeoDataFrame(max_ids_daylight_8, geometry=gs, crs=None)

  radius = radius_base*(test_floor['sun_201803210800_max'].nlargest(ncircles_daylight))
  circle = gdf_daylight_8_area.centroid.buffer(radius, resolution=16)

  gc = gpd.GeoSeries(circle)
  gdf_circles_daylight_8 = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
  identity_2 = gdf_circles_daylight_8.overlay(floor, how='identity')
  ax = identity_2.plot(ax=ax, alpha= 0.5, color = 'Yellow',figsize=(10,10))

  #Daylight 16
  max_ids_daylight_16 = test_floor.iloc[test_floor['sun_201803211600_max'].argsort()[-ncircles_daylight:]]
  max_ids_daylight_16 = max_ids_daylight_16.sort_values(by= 'sun_201803211600_max', ascending=False)
  gdf_daylight_16_area = gpd.GeoDataFrame(max_ids_daylight_16, geometry=gs, crs=None)

  radius = radius_base*(test_floor['sun_201803211600_max'].nlargest(ncircles_daylight))
  circle = gdf_daylight_16_area.centroid.buffer(radius, resolution=16)

  gc = gpd.GeoSeries(circle)
  gdf_circles_daylight_16 = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
  identity_3 = gdf_circles_daylight_16.overlay(floor, how='identity')
  ax = identity_3.plot(ax=ax, alpha= 0.5, color = 'Yellow',figsize=(10,10))

  #Greenery
  max_ids_view_greenery = test_floor.iloc[test_floor['view_greenery_max'].argsort()[-ncircles_greenery:]]
  max_ids_view_greenery = max_ids_view_greenery.sort_values(by= 'view_greenery_max', ascending=False)
  gdf_view_greenery_area = gpd.GeoDataFrame(max_ids_view_greenery, geometry=gs, crs=None)

  radius = radius_base*(test_floor['view_greenery_max'].nlargest(ncircles_greenery))
  circle = gdf_view_greenery_area.centroid.buffer(radius, resolution=16)

  gc = gpd.GeoSeries(circle)
  gdf_circles_greenery = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
  identity_4 = gdf_circles_greenery.overlay(floor, how='identity')
  ax = identity_4.plot(ax=ax, alpha= 0.5, color = 'Green',figsize=(10,10))

  #Noise
  max_ids_noise = test_floor.iloc[test_floor['window_noise_traffic_day_max'].argsort()[-ncircles_noise:]]
  max_ids_noise = max_ids_noise.sort_values(by= 'window_noise_traffic_day_max', ascending=False)
  gdf_noise_area = gpd.GeoDataFrame(max_ids_noise, geometry=gs, crs=None)

  radius = radius_base*(test_floor['window_noise_traffic_day_max'].nlargest(ncircles_noise))
  circle = gdf_noise_area.centroid.buffer(radius, resolution=16)

  gc = gpd.GeoSeries(circle)
  gdf_circles_noise = gpd.GeoDataFrame(circle, geometry=gc, crs=None)
  identity_6 = gdf_circles_noise.overlay(floor, how='identity')
  ax = identity_6.plot(ax=ax, alpha= 0.5, color = 'Red',figsize=(10,10))

  identity_7 = floor.overlay(floor, how='identity')
  ax = identity_7.plot(ax=ax, alpha= 0.5, color = 'White',figsize=(10,10))
  floor.plot(ax=ax, facecolor='none');

  fig_name = test_floor['floor_id'].iloc[0]
  ax.figure.savefig(str(fig_name))
  continue
  